### 设备

In [2]:
import torch
a = torch.ones(3, 3)

In [5]:
a.cpu(), a.cpu().device

(tensor([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]),
 device(type='cpu'))

In [7]:
# a.cuda(2), a.cuda(2).device, a.cuda("cuda:2"), a.cuda("cuda:2").device
# 前提是你有相关硬件和包

### 梯度

In [11]:
x = torch.tensor([1.,2.,3.], requires_grad=True) # 注意要使用浮点型，因为求导过程中可能会产生小数
y = x * 2 + x  # 显然， dy/dx = 3 ->[3, 3, 3]
loss = y.sum()

# 反向传播
loss.backward()
loss.grad, y.grad, x.grad

(None, None, tensor([3., 3., 3.]))

如果不是整个tensor参与计算，那么只有索引出来的部分会求导

In [14]:
x = torch.tensor([1.,2.,3.], requires_grad=True)
y = x[1] * 3
y.backward()
x.grad

tensor([0., 3., 0.])

### ReLU

$
ReLU(x) = 
\begin{cases}
x, & x\gt0 \\
0, & x\lt0
\end{cases}
$

对ReLU求导怎么求？其实就是拆成两个式子：

$
\begin{cases}
    f(x) = x, & x\gt0 \\
    f(x) = 0, & x\lt0
\end{cases}
\Rightarrow
\frac{\partial f}{\partial x} =
\begin{cases}
    x' = 1 \\
    0' = 0
\end{cases}
$ 

在BP中，带上上层的导数g，往往就成了g和0

In [16]:
x = torch.tensor([-1.,-2.,3.], requires_grad=True)
y = x[x>0].sum()
y.backward()
x.grad

tensor([0., 0., 1.])

### detach

detach过的参数就成了常量，不参与求导

In [3]:
x = torch.tensor([1., 2., 3.], requires_grad=True)
dx = x.detach()
y = dx * 2 + x  # dx 不参与求导
loss = y.sum()
loss.backward()
print("x.grad is", x.grad, ", x.requires_grad =", x.requires_grad)

x.grad is tensor([1., 1., 1.]) , x.requires_grad = True


In [4]:
x = torch.tensor([1., 2., 3.], requires_grad=True)
y = x * 2 + x
loss = y.sum()
x.detach_()   # inplace，此时x被修改，此时x的requires_grad=False
loss.backward()
print("x.grad is", x.grad, ", x.requires_grad =", x.requires_grad)

x.grad is None , x.requires_grad = False


In [5]:
a = torch.zeros(2, 2, 2, device="cpu")   # 创建全0 FloatTensor，具有2x2x2维度
print("a.is_cuda =", a.is_cuda)          # 判断tensor是否在cuda上，也就是gpu上
print("a.device =", a.device)            # 获取tensor所在的设备 
print("a.grad =", a.grad)                # 获取tensor的梯度，没有梯度则为None
print("a.ndim =", a.ndim)                # 获取tensor的维度数
print("a.dim() =", a.dim())              # 同ndim一样，获取维度数
print("a.ndimension() =", a.ndimension())# 同ndim一样，获取维度数
print("a.numel() =", a.numel())          # 获取tensor的元素数，其实是每个维度相乘
print("a.nelement() =", a.nelement())    # 获取tensor的元素数，同numel()
print("a.element_size() =", a.element_size())   # 获取元素的大小，Byte为单位，指的是单位元素大小，因为是float，所以是4字节

a.is_cuda = False
a.device = cpu
a.grad = None
a.ndim = 3
a.dim() = 3
a.ndimension() = 3
a.numel() = 8
a.nelement() = 8
a.element_size() = 4


In [11]:
a = torch.arange(9).view(3, 3)
a.t(), a.T   # 就如同a.shape, a.size(), 一个属性，一个方法

(tensor([[0, 3, 6],
         [1, 4, 7],
         [2, 5, 8]]),
 tensor([[0, 3, 6],
         [1, 4, 7],
         [2, 5, 8]]))

### 扩展，复制

tensor with singleton dimensions expanded to a larger size.
只能扩展向量，不能扩展矩阵
横向量竖扩展，竖向量横扩展

`size`也可以用别的tensor对象传进去，用`expand_as`方法会读入参的形状

In [39]:
# a = torch.tensor([[1],[2],[3]])
a = torch.tensor([1,2,3])
a.expand(5, 3)   # 可以，竖向叠
a.expand(3)      # 可以，不变
# a.expand(6)    # 不可以，等于不能横向叠？

b = torch.tensor([[1],[2],[3]])
b.expand(3,3)    # 可以，横向叠至少要原向量本身有两个维度
# b.expand(6,1)  # 又不可以了，等于横向量不能横叠，竖向量不能竖叠

c = torch.ones(2, 3)
# c.expand(4, 3)  # 不可以
# c.expand(2, 6)  # 不可以  所以只能拼向量？



### 设备

```python
a.cpu(), a.device   => device(type='cpu')
a.cuda(2), a.device => device(type='cuda', index=2)
a.cuda('cuda:2')    => just like above
a.to('cuda:2')      => 设备间复制
a.cuda(2)           => 同上

# 设置默认的GPU ID
torch.cuda.set_device(2)
a = torch.tensor([1,2], device=torch.device('cuda'))

```

In [40]:
torch.cuda.device_count()

0

In [41]:
torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

AssertionError: Torch not compiled with CUDA enabled